In [16]:
import pandas as pd
import os

In [3]:
DATA_PATH = '../data/meteo_data'
met_files = os.listdir(DATA_PATH)

In [173]:
df = pd.DataFrame()
for file in met_files:
    df_file = pd.read_csv(os.path.join(DATA_PATH, file), index_col=0)
    if 'Unnamed: 0' in df_file.columns:
        df_file = df_file.drop('Unnamed: 0', axis = 1)
    df = pd.concat((df, df_file))
    

df = df.reset_index(drop=True)

In [174]:
df.shape 

(32209, 26)

In [175]:
cols_without_nans = df.isna().sum()[(df.isna().sum() == 0)].index

In [177]:
cols_without_nans

Index(['datetime', 'datetimeEpoch', 'temp', 'feelslike', 'humidity', 'dew',
       'precip', 'precipprob', 'windspeed', 'winddir', 'cloudcover',
       'conditions', 'icon', 'stations', 'source'],
      dtype='object')

In [178]:
df = df[cols_without_nans]

In [180]:
from pytz import timezone


local_time = timezone('Europe/Madrid')
utc_time = timezone('UTC')

df['datetimeEpoch'] = pd.to_datetime(df['datetimeEpoch'], unit = 's')
df['datetimeEpoch'] = df.datetimeEpoch.dt.tz_localize(utc_time)
df['datetimeEpoch'] = df.datetimeEpoch.dt.tz_convert(local_time)
df['datetimeEpoch'] = pd.to_datetime(df.datetimeEpoch)
df['month'] = df.datetimeEpoch.dt.month
df['day'] = df.datetimeEpoch.dt.day
df['hour'] = df.datetimeEpoch.dt.hour
df['year'] = df.datetimeEpoch.dt.year

In [181]:
clean_df = df[(df.year != 2023)].drop_duplicates(keep='first')

In [184]:
(365*2 + 366)*24

26304

In [201]:
clean_df

<DstTzInfo 'Europe/Madrid' CET+1:00:00 STD>

In [204]:
(
    (clean_df.month.value_counts().sort_index() == 24*31*3) 
    | (clean_df.month.value_counts().sort_index() == 24*30*3) 
    | (clean_df.month.value_counts().sort_index() == 24*(28*2+29))
    )

1      True
2      True
3     False
4      True
5      True
6      True
7      True
8      True
9      True
10    False
11     True
12     True
Name: month, dtype: bool

In [207]:
clean_df.datetimeEpoch

0       2020-01-01 00:00:00+01:00
1       2020-01-01 01:00:00+01:00
2       2020-01-01 02:00:00+01:00
3       2020-01-01 03:00:00+01:00
4       2020-01-01 04:00:00+01:00
                   ...           
29948   2022-12-31 19:00:00+01:00
29949   2022-12-31 20:00:00+01:00
29950   2022-12-31 21:00:00+01:00
29951   2022-12-31 22:00:00+01:00
29952   2022-12-31 23:00:00+01:00
Name: datetimeEpoch, Length: 26306, dtype: datetime64[ns, Europe/Madrid]

In [209]:
DATA_PATH

'../data/meteo_data'

In [210]:
clean_df.to_csv(DATA_PATH + '/cleaned_meteo_data_until_2022.csv')